## Trimming Characters from Strings
Let us go through how to trim unwanted characters using Spark Functions.

In [1]:
%%HTML
<iframe width="560" height="315" src="https://www.youtube.com/embed/qWv8zfSOPZM?rel=0&amp;controls=1&amp;showinfo=0" frameborder="0" allowfullscreen></iframe>

* We typically use trimming to remove unnecessary characters from fixed length records.
* Fixed length records are extensively used in Mainframes and we might have to process it using Spark.
* As part of processing we might want to remove leading or trailing characters such as 0 in case of numeric types and space or some standard character in case of alphanumeric types.
* As of now Spark trim functions take the column as argument and remove leading or trailing spaces. However, we can use `expr` or `selectExpr` to use Spark SQL based trim functions to remove leading or trailing spaces or any other such characters.
  * Trim spaces towards left - `ltrim`
  * Trim spaces towards right - `rtrim`
  * Trim spaces on both sides - `trim`

Let us start spark context for this Notebook so that we can execute the code provided. You can sign up for our [10 node state of the art cluster/labs](https://labs.itversity.com/plans) to learn Spark SQL using our unique integrated LMS.

In [1]:
from pyspark.sql import SparkSession

import getpass
username = getpass.getuser()

spark = SparkSession. \
    builder. \
    config('spark.ui.port', '0'). \
    config("spark.sql.warehouse.dir", f"/user/{username}/warehouse"). \
    enableHiveSupport(). \
    appName(f'{username} | Python - Processing Column Data'). \
    master('yarn'). \
    getOrCreate()

If you are going to use CLIs, you can use Spark SQL using one of the 3 approaches.

**Using Spark SQL**

```
spark2-sql \
    --master yarn \
    --conf spark.ui.port=0 \
    --conf spark.sql.warehouse.dir=/user/${USER}/warehouse
```

**Using Scala**

```
spark2-shell \
    --master yarn \
    --conf spark.ui.port=0 \
    --conf spark.sql.warehouse.dir=/user/${USER}/warehouse
```

**Using Pyspark**

```
pyspark2 \
    --master yarn \
    --conf spark.ui.port=0 \
    --conf spark.sql.warehouse.dir=/user/${USER}/warehouse
```

### Tasks - Trimming Strings

Let us understand how to use trim functions to remove spaces on left or right or both.
* Create a Dataframe with one column and one record.
* Apply trim functions to trim spaces.

In [2]:
l = [("   Hello.    ",) ]

In [3]:
df = spark.createDataFrame(l).toDF("dummy")

In [4]:
df.show()

+-------------+
|        dummy|
+-------------+
|   Hello.    |
+-------------+



In [5]:
from pyspark.sql.functions import col, ltrim, rtrim, trim

In [6]:
df.withColumn("ltrim", ltrim(col("dummy"))). \
  withColumn("rtrim", rtrim(col("dummy"))). \
  withColumn("trim", trim(col("dummy"))). \
  show()

+-------------+----------+---------+------+
|        dummy|     ltrim|    rtrim|  trim|
+-------------+----------+---------+------+
|   Hello.    |Hello.    |   Hello.|Hello.|
+-------------+----------+---------+------+



In [7]:
from pyspark.sql.functions import expr

In [8]:
spark.sql('DESCRIBE FUNCTION rtrim').show(truncate=False)

+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|function_desc                                                                                                                                                                                         |
+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|Function: rtrim                                                                                                                                                                                       |
|Class: org.apache.spark.sql.catalyst.expressions.StringTrimRight                                                                                                                                   

In [9]:
# if we do not specify trimStr, it will be defaulted to space
df.withColumn("ltrim", expr("ltrim(dummy)")). \
  withColumn("rtrim", expr("rtrim('.', rtrim(dummy))")). \
  withColumn("trim", trim(col("dummy"))). \
  show()

+-------------+----------+--------+------+
|        dummy|     ltrim|   rtrim|  trim|
+-------------+----------+--------+------+
|   Hello.    |Hello.    |   Hello|Hello.|
+-------------+----------+--------+------+



In [10]:
spark.sql('DESCRIBE FUNCTION trim').show(truncate=False)

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|function_desc                                                                                                                                                                                                                                                                                                                                                                  |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [11]:
df.withColumn("ltrim", expr("trim(LEADING ' ' FROM dummy)")). \
  withColumn("rtrim", expr("trim(TRAILING '.' FROM rtrim(dummy))")). \
  withColumn("trim", expr("trim(BOTH ' ' FROM dummy)")). \
  show()

+-------------+----------+--------+------+
|        dummy|     ltrim|   rtrim|  trim|
+-------------+----------+--------+------+
|   Hello.    |Hello.    |   Hello|Hello.|
+-------------+----------+--------+------+

